In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [4]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

cities_path = "../WeatherPy/cities_weather.csv"

# Read School and Student Data File and store into Pandas DataFrames
cities_data = pd.read_csv(cities_path)

#Create the dataframe
cities_data_df = pd.DataFrame(cities_data)
cities_data_df.head()

,Cities,Countries,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Date
0,Bakel,SN,14.90,-12.46,72.5,20,80,10.5,02-09-2021
1,Jamestown,SH,-15.94,-5.72,68.4,88,54,15.0,02-09-2021
2,Nikolskoye,RU,59.70,30.79,5.0,85,90,2.2,02-09-2021
3,Arraial Do Cabo,BR,-22.97,-42.03,72.4,73,100,2.1,02-09-2021
4,Port Hardy,CA,50.70,-127.42,37.4,74,1,8.0,02-09-2021


In [11]:
#Humidity Heatmap

#Configure gmaps
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_data_df[["Latitude", "Longitude"]]
weights = cities_data_df["Humidity (%)"]

#Create map
figure_layout = {'width': '800px',
                 'height': '600px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=1)

In [12]:
#Add Heatmap layer to map.
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…